In [222]:
import pandas as pd

In [223]:
#Leggo il file excel TdB
df = pd.read_excel('/Users/federico/Desktop/TableauDeBoard/Tableau de Bord_Interim Master 30.11.2023.xlsb', sheet_name='TdB - Executive per ptf.',  skiprows=7)

In [224]:
#Estendo la visualizzazione del dataframe su righe e colonne
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [225]:
#droppa le colonne che hanno nome Unnamed
col_index = [index for index, col in enumerate(df.columns) if 'Unnamed' in str(col)]
df.drop(df.columns[col_index], axis=1, inplace=True)

#imposta il valore di df.loc[0, 'Current period:\n2023;Q4;November 30 th'] a NaN
df.loc[0, 'Current period:\n2023;Q4;November 30 th'] = pd.NA



#individua la riga delle Note e droppa dall'indice di quella riga in poi
row_index = df[df['Current period:\n2023;Q4;November 30 th'] == 'Notes: '].index
print(row_index)
df.drop(df.index[row_index[0]:], inplace=True)
df.reset_index(drop=True, inplace=True)

Index([261], dtype='int64')


In [226]:
#Se individuo due righe consecutive che contengono NaN, allora ne droppo una
indeces_to_drop = []
count = 0
for i, value in enumerate(df['Current period:\n2023;Q4;November 30 th']):
    if pd.isna(value) == False:
        count = 0
    else:
        count += 1
        if count == 2:
            indeces_to_drop.append(i)

df.drop(indeces_to_drop, inplace=True)
df.reset_index(drop=True, inplace=True)



In [227]:
def auto_segmentation(df):
    indici_nan = df.index[df['Current period:\n2023;Q4;November 30 th'].isna()].tolist()
    segmenti = {}

    for start, end in zip(indici_nan, indici_nan[1:] + [None]):
        segmento = df.iloc[start+1:end]
        if not segmento.empty:
            nome_df = str(segmento.iloc[0, 0]).replace(' ', '_').replace(':', '').replace(';', '_').replace('\n', '_').replace('.', '').replace('(', '').replace(')', '').replace('-', '_').replace('/', '_')
            segmenti[nome_df] = segmento

    return segmenti

segmenti = auto_segmentation(df)
lista_df = []
for i in segmenti.keys():
    exec(f'global df_{i}; df_{i} = segmenti[i]', globals())
    exec(f'lista_df.append(df_{i})', globals())
    exec(f'df_{i}.reset_index(drop=True, inplace=True)', globals())


RIMARREBBE INFINE DA SCARTARE LA CELLA RIASSUNTIVA PER I VARI SEGMENTI DI ALICUDI.

UN ALTRO MODO DI FARE UN CHECK POTREBBE ESSERE QUELLO DI LEGGERE TUTTO E DI FARE UNA LEFT JOIN SU: NOME RIGA + IMPORTO

In [271]:
def match_clusters(df):
    i = 0
    k = 0
    tries = 0
    clusters = []

    while i < len(df):
        # Valore di partenza per il cluster
        valore_iniziale = df.iloc[i]['# Borrowers']
        somma = valore_iniziale
        j = i + 1

        # Calcola la differenza cumulativa finché non raggiunge zero o la fine del DataFrame
        while j < len(df) and somma != 0:
            somma -= df.iloc[j]['# Borrowers']
            j += 1
            tries += 1

        # Se la somma è zero, allora abbiamo trovato un cluster
        if len(df) == 1:
            clusters.append(df.iloc[0])
            break
        if somma == 0:
            # Aggiungi il cluster al risultato
            cluster = df.iloc[i:j]
            clusters.append(cluster)
            i = j-1
            if k == 0:
                j -= 1
                if j == len(df) -1:
                    if i < len(df) - 1:
                        clusters.append(df.iloc[-1])
                    else:
                        break
            if j != i+2 and i != 0:
                i = j
        elif tries > len(df):
            clusters.append(df.iloc[-1])
            break
        else:
            k += 1
            i = k
        

    # Utilizzo l'hashing per identificare i dataframes duplicati
    # Sommo gli hashing per ottenere un unico valore per ogni DataFrame
    clusters_hashed = [pd.util.hash_pandas_object(x, index=True).sum() for x in clusters]

    # Creiamo un set vuoto per tenere traccia degli hash unici
    unique_hashes = set()
    unique_clusters = []

    for i, hash_value in enumerate(clusters_hashed):
        if hash_value not in unique_hashes:
            unique_clusters.append(clusters[i])
            unique_hashes.add(hash_value)

    # Ora unique_clusters contiene i DataFrame senza duplicati
    clusters = unique_clusters



    return clusters



In [272]:
# Inserisco in una lista i risultati della clusterizzazione su tutti i DataFrame
lista_clusters = []
for i in range(len(lista_df)):
    lista_clusters.append(match_clusters(lista_df[i]))

In [273]:
all_clusters = []
for cluster in lista_clusters:
    if len(cluster) > 1:
        for sub_cluster in cluster:
            all_clusters.append(sub_cluster)
    else:
        all_clusters.append(cluster)

for index, cluster in enumerate(all_clusters):
    if len(cluster) == 0:
        all_clusters.pop(index)

In [274]:
def get_title(df):
    if isinstance(df, pd.DataFrame):
        # Prima resetta l'indice del DataFrame
        df.reset_index(drop=True, inplace=True)
        # Ora accedi alla prima riga in modo sicuro
        title = df.loc[0, 'Current period:\n2023;Q4;November 30 th']
        print(title)
        return title
    elif isinstance(df, pd.Series):
        title = df['Current period:\n2023;Q4;November 30 th']
        print(title)
        return title
    else:
        return get_title(df[0])


In [ ]:
titles_list = []
for index, df in enumerate(all_clusters):
    titles_list.append(get_title(df))

In [280]:
titles_list[0:30]

['Total Full Portfolios',
 'Total Full Ptf.s Intrum Italy',
 'Total Proprietary',
 'Total Proprietary Revalue',
 'Revalue',
 'Other Clients',
 'o.w. Phone&Field Intrum Italy',
 'Total Proprietary Intrum Italy',
 'Total Proprietary Direct',
 'Kenobi',
 'Intrum - Saturnia',
 'Intrum - West',
 'Intrum - Elettra',
 'Caterina',
 'Jawa',
 'Fantino',
 'IJDF Italy',
 'Intrum Debt AG',
 'LSF WEST - REOCO',
 'Arizona SPV Mandalorian',
 'Arizona Oasis',
 'Total Propriatery JV',
 'TOP',
 'Large',
 'Medium',
 'Small Secured',
 'Small Unsecured',
 'Not segmented',
 'Savoy Reoco',
 'Portland']

In [ ]:
i = 0
j = 0
while i < len(lista_df):
    df1 = lista_df[i]
    df2 = all_clusters[j]
    joined_df = pd.merge(df1, df2, how='inner')
    if len(joined_df) == len(df1):
        j += 1
        cluster = 
    else:
        i += 1


        



